In [1]:
# Purpose: Quickly analysing ENSOWind EXP to see if all okay
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #                         
#     Maurice F. Huguenin, m.huguenin-virchaux@unsw.edu.au, 28. 09. 2021      #                                                
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

# % preamble ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #                         
base = '/g/data/e14/mv7494/access-om2/archive/025deg_jra55_iaf/'
save = '/g/data/e14/mv7494/'

import xarray as xr # for working with labelled multi-dimensional arrays                                                                
import numpy as np # for numerical operations                                                           
import cmocean as cm
from datetime import datetime # package for printing time in plot                                                                                                       
import matplotlib.pyplot as plt # for matlab-like plotting                                              
import cartopy.crs as ccrs # for maps     
import itertools
import cosima_cookbook as cc
import pandas as pd
from matplotlib import gridspec
from cdo import *
cdo = Cdo()
import os
import sys
import warnings
import matplotlib.path as mpath
import time
import matplotlib.animation as animation # library that makes the movie
import matplotlib.patheffects as PathEffects
warnings.filterwarnings('ignore') # suppress warnings
from matplotlib import ticker
import nc_time_axis
import cftime
import cartopy.mpl.ticker as cticker
import string # for subplot annotation
import cartopy.feature as cfeature
from matplotlib import gridspec
import bottleneck as bn # for time series moving average
from scipy import stats # create linear trend over the 3rd to 5th cycles for the upper 2000 m

#import warnings
#warnings.filterwarnings('ignore') # suppress warnings

import nc_time_axis
import cftime

# for displaying multiple output in one cell
from IPython.display import display as dp

from dask.distributed import Client
client = Client(n_workers=16)        
client
rotblau = [[0.4 , 0.  , 0.12], [0.47, 0.02, 0.13], [0.62, 0.07, 0.16], [0.66, 0.16, 0.2],
           [0.8 , 0.3 , 0.27], [0.87, 0.44, 0.35], [0.93, 0.58, 0.45], [0.96, 0.7 , 0.58], 
           [0.98, 0.8 , 0.71], [0.98, 0.94, 0.92], [0.93, 0.95, 0.96], [0.85, 0.91, 0.95],
           [0.76, 0.87, 0.93], [0.64, 0.8 , 0.89], [0.49, 0.72, 0.84], [0.34, 0.62, 0.79], 
           [0.23, 0.53, 0.74], [0.16, 0.44, 0.7] , [0.1 ,0.35, 0.6], [0.05, 0.24, 0.45], [0.02, 0.19, 0.38]]

### Using the cosima cookbook to load in daily sea surface velocity maps

In [ ]:
%%time
session = cc.database.create_session()
import matplotlib.path as mpath 
# -------------------------------------------------------------------------------------------- #
NA_lat        = [    60,    361] # [    60,    361] = 75°N - 0°                                #
NA_lon        = [   320,    801] # [   320,    801] = 100°W - 20°E                             #
time          = [350650, 613633] # [350650, 613633] = 1980-01-01T00:00:00, 2010-01-01T01:00:00 #
# -------------------------------------------------------------------------------------------- #
filename = ['/g/data/e14/mv7494/ERA5/ERA5_NorthAtlantic_nsf_clim_daily_1980-01-01_to_2010-08-01.nc',
            '/g/data/e14/mv7494/ERA5/ERA5_NorthAtlantic_nsf_daily_2023-01-01_to_2023-08-01.nc']
era_path = '/g/data/rt52/era5/single-levels/reanalysis/'
vars = ['slhf', 'sshf', 'ssrd', 'strd'] # surface latent heat flux, surface sensible heat flux, 
#       'slhf' Surface latent heat flux
#               'sshf' Surface sensible heat flux
#                       'ssrd' Surface short-wave (solar) radiation downwards
#                              'strd' Surface long-wave (thermal) radiation downwards

for e in range(2): # loop through the climatology and the 2023 year
    if e == 0: era_short = '/2023/*.nc'; suffix = 'nsf_clim'    # select all the years
    if e == 1: era_short = '/2023/*.nc'; suffix = 'nsf'         # only select year 2023
        
#    if not os.path.exists(filename[e]):
#        print('Calculating anomalies as they are not yet done')
#        print('----------------------------------------------') 

    for f in range(1): # loop through the four components of the net surface heat flux
        data = xr.open_mfdataset(era_path + vars[f] + era_short,
                                 chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}
                                )[vars[f]][:,NA_lat[0]:NA_lat[1],NA_lon[0]:NA_lon[1]]#.resample(time='D').mean()
        if f == 0: slhf = data
        if f == 1: sshf = data
        if f == 2: ssrd = data
        if f == 3: strd = data
    nsf = (slhf + sshf + ssrd + strd).to_dataset(name=suffix).to_netcdf(filename[e])  
# Wall time: 1min 24 s for one year

In [5]:
# ----------------------------------------------------------------------------------------------------------------------- #
#                                                   Notes down below                                                      #
# ----------------------------------------------------------------------------------------------------------------------- #